In [2]:
import pandas as pd
import numpy as np

from tqdm import tqdm

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.base import clone

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer


import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import skew, ks_2samp # Kolmogorov-Smirnov Test

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error, mean_absolute_error, r2_score \
                            , precision_recall_fscore_support, log_loss

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

import numpy as np

In [4]:
a = [1,2,3]
np.array(a).mean()

2.0

In [10]:
import seaborn as sns

df = sns.load_dataset('titanic')

In [2]:
class LSTMTagger(torch.nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = torch.nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = torch.nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [3]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    # Tags are: DET - determiner; NN - noun; V - verb
    # For example, the word "The" is a determiner
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}  # Assign each tag with a unique index

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [6]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-1.0500, -0.9256, -1.3714],
        [-1.0188, -0.9739, -1.3419],
        [-1.1330, -0.9662, -1.2126],
        [-1.1818, -0.9763, -1.1501],
        [-1.0766, -0.9916, -1.2439]])
tensor([[-0.3892, -1.2426, -3.3890],
        [-2.1082, -0.1328, -5.8464],
        [-3.0852, -5.9469, -0.0495],
        [-0.0499, -3.4414, -4.0961],
        [-2.4540, -0.0929, -5.8799]])


In [ ]:
def capture_gradient(data_loader, model, loss_func):
    gradients = []
    for inputs, labels in data_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()

        # Collect gradients
        for param in model.parameters():
            gradients.append(param.grad.view(-1))

    total_gradients = torch.cat(gradients)
    return total_gradients

# Example usage:
train_gradient = capture_gradient(DataLoader, model, criterion)

In [ ]:
def calculate_score_base_on_metric(y_pred, y_true, metric):
    supported = [["MSE", "MAE", "r2"],["accuracy", "precision", "recall", "f1"], ["log_loss"]]
    if metric == "MSE":
        return mean_squared_error(y_true, y_pred)
    elif metric == "MAE":
        return mean_absolute_error(y_true, y_pred)
    elif metric == "r2":
        return r2_score(y_true, y_pred)
    elif metric == "accuracy":
        return accuracy_score(y_true, y_pred)
    elif metric == "precision":
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
        return precision
    elif metric == "recall":
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
        return recall
    elif metric == "f1":
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
        return f1
    elif metric == "log_loss":
        return log_loss(y_true, y_pred)
    else:
        raise KeyError("Unsupported Metric")
    
def calculate_influence_base_on_metric(base_score, current_score, metric):
    # Errors: lower better
    if metric in ["MSE", "MAE", "log_loss"]:
        influence = current_score - base_score
    # Accuracy: higher better
    elif metric in ["accuracy", "precision", "recall", "f1", "r2"]:
        influence = base_score - current_score
    else:
        raise KeyError("Unsupported Metric")
    return influence
        

In [ ]:
class InfluenceAnalyze_PT():
    def __init__(self, data_loader, model, loss_func):
        self.data_loader = data_loader
        self.model = model

        self.data_category = "tabular"

        self.data_influences = np.zeros(len(X))
        self.feature_influences = None
        self.influence_method = None

        self.supported_tasks = ["regression", "classification", "probabilities"]
        self.task = task
        if task not in self.supported_tasks:
            print("Supported tasks:", self.supported_tasks)
            raise KeyError("Unsupported Task")
        
        self.supported_metrics = [["MSE", "MAE", "r2"],["accuracy", "precision", "recall", "f1"], ["log_loss"]]
        self.metric = None

        if metric:
            if metric not in self.supported_metrics[self.supported_tasks.index(task)]:
                print(f"Supported metrics for {task}:", self.supported_metrics[self.supported_tasks.index(task)])
                raise KeyError("Unsupported Metric for this task")
            else:
                self.metric = metric
        else:
            # Default chose the first metric
            self.metric = self.supported_metrics[self.supported_tasks.index(task)][0]

        self.preprocess_pipeline = None

        model.fit(X, y)
        y_pred = model.predict(self.X)
        self.base_score = calculate_score_base_on_metric(y_pred, y, self.metric)

        print(f"Data size, X: {X.shape}, y: {y.shape}")
        print(f"Task: {self.task}, using metric: {self.metric}")
        print(f"Base score: {self.base_score}")
    
    def help(self):
        print("This calculate Influence for data")
        print("Provide data as X and target as y")
        print("Supported influence methods: LOO, shapley")
        print("Supported influence metrics: ")

    def Feature_analyze(self, stat=True):
        """
        This extracts each feature column to analyze the influence of each column.
        """
        print("Analyzing each features")

        X = self.X
        y = self.y
        n_features = X.shape[1]
        if n_features <= 1:
            print("Data only has one feature.")
            return
        # Calculate the base accuracy with all features
        model = self.model

        self.feature_influences = np.zeros(n_features)

        for i in tqdm(range(n_features)):
            X_droped = self.X.drop(self.X.columns[i], axis=1)

            # fit model
            model.fit(X_droped, y)
            # calculate the accuracy difference as influence
            y_pred = model.predict(X_droped)

            current_score = calculate_score_base_on_metric(y_pred, y, self.metric)
            influence = calculate_influence_base_on_metric(self.base_score, current_score, self.metric)
            self.feature_influences[i] = influence

        if stat:
            for i in range(n_features):
                print(f"Column: {X.columns[i]}, influence: {self.feature_influences[i]:.4f}")

            min_feature = self.X.iloc[:, [self.feature_influences.argmin()]]

            if self.feature_influences.min() >= 0:
                print("All features have positive impact")
                return

            print(f"The feature has the worst influence: {X.columns[self.feature_influences.argmin()]}, with {self.metric} impact: {self.feature_influences.min()*100:.2f}%")
            print("Skewness of the feature:", skew(min_feature))

            sns.histplot(min_feature, kde=True)  # The `kde` parameter adds a Kernel Density Estimate plot over the histogram.
            plt.title('Distribution')
            plt.xlabel(X.columns[self.feature_influences.argmin()])
            plt.ylabel('Frequency')
            plt.show()
            
            

    def PrintInfluence(self):
        if self.influence_method:
            print("The data last used:", self.influence_method)
            print(self.data_influences)
            print("Average influence:", self.data_influences.mean())
            print("Worst influence:", self.data_influences.min(), ", index:", self.data_influences.argmin())
            print("The data with min influence:")
            print(self.X.iloc[self.data_influences.argmin()])
        else:
            print("No analysis has been done")

    def CalculateInfluence(self, method='LOO', n_random_row=10, num_shuffles=10, threshold=0.97, seed=1, stat=True):
        n_random_row = len(self.X) if n_random_row > len(self.X) or n_random_row < 0 else n_random_row
        if method == 'LOO':
            self.LOOinfluence(n_random_row=n_random_row, seed=seed, stat=stat)
        elif method == 'shapley':
            self.shapley_influence(num_shuffles=num_shuffles, threshold=threshold, seed=seed, stat=stat)
        else:
            print("Invalid method")

    def LOOinfluence(self, n_random_row, seed=42, stat=True):
        # Clear influences
        self.data_influences = np.zeros(len(self.X))

        print("Calculating data influence using Leave One Out")
        # To select 10 random row indexs for LOO
        np.random.seed(seed)

        selected_indices = np.random.choice(len(self.X), n_random_row, replace=False)

        influences = {}
        # Calculate the base accuracy with all data points
        model = self.model

        # Exclue each random row to compute the LOO prediction
        for loo_ix in tqdm(selected_indices):
            # split data
            X_train_loo = self.X.drop(X.index[loo_ix])
            y_train_loo = np.delete(self.y, loo_ix)
            # fit model
            model.fit(X_train_loo, y_train_loo)
            # calculate the accuracy difference as influence
            y_pred = model.predict(X)
            current_score = calculate_score_base_on_metric(y_pred, y, self.metric)
            influence = calculate_influence_base_on_metric(self.base_score, current_score, self.metric)

            influences[loo_ix] = influence
            self.data_influences[loo_ix] = influence

        self.influence_method = 'LOO'
        if stat:
            self.PrintInfluence()

        return influences

    def shapley_influence(self, num_shuffles=10, threshold=0.97, seed=42, stat=True):
        """
        TMC based shapley inflence calculation
        """
        # Clear influences
        self.data_influences = np.zeros(len(self.X))

        print("Calculating data influence using Shapley Value")
        X = self.X
        y = self.y
        model = self.model
        np.random.seed(seed)

        N = X.shape[0]
        full_performance = self.base_score

        shapley_values = np.zeros(N)
        for j in tqdm(range(num_shuffles)):
            permutation = np.random.permutation(N)
            X_perm = X.iloc[permutation]
            y_perm = y[permutation]

            prev_performance = 0
            for i in tqdm(range(N)):
                model.fit(X_perm[:i+1], y_perm[:i+1])
                performance = accuracy_score(y, model.predict(X))
                marginal_contribution = performance - prev_performance
                shapley_values[permutation[i]] += marginal_contribution
                prev_performance = performance

                if performance >= threshold * full_performance:
                    break

        self.data_influences = shapley_values / num_shuffles
        self.method = 'shapley'
        if stat:
            self.PrintInfluence()

        return self.data_influences
    
    def Analyze_data_influence(self, plot=True, negative_threshold=0.15):
        if not self.influence_method:
            print("No data influence computation has been done, returning.")
            return

        data_influences = self.data_influences
        X = self.X
        y = self.y


        negative_size = (data_influences < 0).sum()

        if negative_size < negative_threshold*X.shape[0]:
            print("The dataset contains mostly potive data, returning.")
            return

        negative_data_points = X[data_influences < 0]
        negative_targets = y[data_influences < 0]

        features = X.columns
        n_features = len(features)

        if plot:
            n_cols = int(n_features**0.5)
            n_rows = (n_features + n_cols - 1) // n_cols

            fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, n_rows * 5))  # Adjust size as needed
            fig.suptitle('Feature Distributions for Overall and Negative Data Points')

            # Flatten the axes array if necessary (for easy indexing)

            if n_rows > 1:
                axes = axes.flatten()
            else:
                axes = [axes]

            # Loop through the features and plot histograms
            for idx, feature in enumerate(features):
                # Select the current axis
                ax = axes[idx]
                # Histogram for the overall dataset
                ax.hist(X[feature].dropna(), bins=20, alpha=0.5, label='Overall', color='blue')  # Ensure to drop NA values
                # Histogram for the negative data points
                ax.hist(negative_data_points[feature].dropna(), bins=20, alpha=0.5, label='Negative', color='red')
                ax.set_title(feature)
                ax.set_xlabel(feature)
                ax.set_ylabel('Frequency')
                ax.legend()

            # Hide any unused axes if the number of features is odd
            if n_features % n_cols != 0:
                for ax in axes[n_features:]:
                    ax.axis('off')

            plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust the rectangle in which to fit the subplots
            plt.show()

        print("Testing distribution different amoung negative data points and original dataset.")
        for feature in features:
            # Perform Anderson-Darling test to test if the distribution of that feature is aligned 
            ks_stat, ks_pvalue = ks_2samp(X[feature].dropna(), negative_data_points[feature].dropna(), method='exact')

            # print(f"Feature: {feature}, KS Statistic: {ks_stat}, P-value: {ks_pvalue}")

            if ks_pvalue < 0.1:
                unbalanced_feature = negative_data_points[feature]
                print(f"Feature {feature} distributions is statistically different.")
                print(f"Consider examine data with feature {feature} with range {unbalanced_feature.mean() - unbalanced_feature.std():.3f} to {unbalanced_feature.mean() + unbalanced_feature.std():.3f}")

                plt.subplot(1, 2, 1)
                plt.hist(X[feature], bins=20, alpha=0.5, label='Overall', color='blue')
                plt.hist(negative_data_points[feature], bins=20, alpha=0.5, label='Negative', color='red')
                plt.title(f'{feature} Distribution')
                plt.xlabel(feature)
                plt.ylabel('Frequency')
                plt.legend()
    
    def Auto_preprocess(self):
        """
        This auto analyze data points and feature to suggestion an optimal pipeline for dataprocessing
        """
        X = self.X
        y = self.y
        model = self.model
        
        if not self.feature_influences:
            self.Feature_analyze(stat=False)

        negative_features = X[X.columns[self.feature_influences < 0]]
        
        numeric_features = negative_features.select_dtypes(include=['int', 'float']).columns

        categorical_features = negative_features.select_dtypes(include=['object', 'category']).columns
        preprocessing_steps = []

        current_base_score = self.base_score

        # Numeric Feature Preprocessing
        for feature in numeric_features:
            current_steps = preprocessing_steps
            if X[feature].isnull().mean() > 0.1:  # Arbitrary threshold for missing data
                preprocessing_steps.append((f'imputer_{feature}', SimpleImputer(strategy='median'), [feature]))
                current_base_score = self.try_adding_preprocess(preprocessing_steps, current_base_score, column=feature, method='imputer')
                
            if X[feature].skew() > 1 or X[feature].skew() < -1:  # Check skewness
                preprocessing_steps.append((f'scaler_{feature}', PowerTransformer(method='yeo-johnson'), [feature]))
                current_base_score = self.try_adding_preprocess(preprocessing_steps, current_base_score, column=feature, method='scaler')

            # None of the method works try removing
            if current_steps == preprocessing_steps:
                print(f"None of the preprocess works for this column: {feature}. Consier removing it or examine it")
                

        # Categorical Feature Preprocessing
        for feature in categorical_features:
            current_steps = preprocessing_steps
            if X[feature].nunique() > 10:  # Arbitrary cutoff for too many categories
                preprocessing_steps.append((f'encoder_{feature}', OneHotEncoder(handle_unknown='ignore'), [feature]))
                current_base_score = self.try_adding_preprocess(preprocessing_steps, current_base_score, column=feature, method='encoder')

            if X[feature].isnull().mean() > 0.1:
                preprocessing_steps.append((f'imputer_{feature}', SimpleImputer(strategy='constant', fill_value='missing'), [feature]))
                current_base_score = self.try_adding_preprocess(preprocessing_steps, current_base_score, column=feature, method='imputer')

            # None of the method works try removing
            if current_steps == preprocessing_steps:
                print(f"None of the preprocess works for this column: {feature}. Consier removing it or examine it")

        # Create the column transformer and pipeline
        preprocessor = ColumnTransformer(transformers=preprocessing_steps, remainder='passthrough')
        full_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', clone(model))])

        # Fit the pipeline
        full_pipeline.fit(X, y)

        y_pred = full_pipeline.predict(X)
        current_score = calculate_score_base_on_metric(y_pred, y, self.metric)

        preprocess_influence = -calculate_influence_base_on_metric(self.base_score, current_score, self.metric)

        print(f"Preprocess pipeline: {preprocessing_steps}")
        print(f"New score {current_score}, with improvement {preprocess_influence}")

        self.preprocess_pipeline = full_pipeline

        return full_pipeline

    def try_adding_preprocess(self, preprocessing_steps, current_base_score, column, method):
        print(f"Trying {method} on column: {column}")
        preprocessor = ColumnTransformer(transformers=preprocessing_steps, remainder='passthrough')
        temp_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', clone(self.model))])
        # Fit the pipeline
        temp_pipeline.fit(self.X, self.y)
        y_pred = temp_pipeline.predict(self.X)
        current_score = calculate_score_base_on_metric(y_pred, y, self.metric)
        preprocess_influence = -calculate_influence_base_on_metric(current_base_score, current_score, self.metric)
        print(f"This preprocess has influence: {preprocess_influence}")
        if preprocess_influence > 0:
            print("Performance Improved, saved this preprocess")
            return current_score
        else:
            print("Preprocess dones't work")
            preprocessing_steps.pop()
            return current_base_score